In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models


In [ ]:
import matplotlib.pyplot as plt

# Pick a sample from the test dataset
image, label = test_data[42]  # You can change the index
plt.imshow(image.permute(1, 2, 0) * 0.5 + 0.5)  # Unnormalize for display
plt.title(f"Actual: {class_names[label]}")
plt.axis('off')
plt.show()

# Prepare image for model input
image = image.unsqueeze(0).to(device)

# Predict
with torch.no_grad():
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)
    print(f"Predicted: {class_names[predicted.item()]}")


NameError: name 'test_data' is not defined

In [3]:
# Data Preprocessing for ResNet
transform = transforms.Compose([
    transforms.Resize(224),   # Resize to 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1,1]
])

In [11]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [10]:
# Load and Modify ResNet18
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

resnet = models.resnet18(pretrained=True)
for param in resnet.parameters():
    param.requires_grad = False # Freeze all layers except the final one

resnet.fc=nn.Linear(512,10)  # Replace final layer for 10 classes
resnet = resnet.to(device)


/home/raj/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/raj/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)

In [3]:
# Training Loop
for epoch in range(5):
    resnet.train()  
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader):.4f}")



NameError: name 'resnet' is not defined

In [ ]:
# To know the layers of a model
#import torch
#import torchvision.models as models
#resnet = models.resnet18(weights=None)  # weights=None means random initialization
#print(resnet)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [2]:
import torch
torch.save(resnet.state_dict(), 'resnet18_cifar10.pth')

NameError: name 'resnet' is not defined

In [4]:
# Evaluation of ResNet18 on ImageNet
resnet.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


NameError: name 'resnet' is not defined

In [ ]:
# Load Later and Predict
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Step : Recreate the same architecture
model =models.resnet18(weights=None)
model.fc = nn.Linear(512,10)  # Match your trained final layer
model.load_state_dict(torch.load('resnet18_cifar10.pth', map_location=device))
model = model.to(device)
model.eval()

